# Plots
This file demonstrates how plots for the article titled "Model guided design of enhanced bi-stable controllers to  effectively switch cellular states" (Raj K., Wong W. T. Z., Zhang B., & Mahadevan R.) (2022) were generated

In [2]:
from dynaCA.core import DynaModel, SwitchModel
from dynaCA.plotting import *
from dynaCA.core.DynaTrajectory import *
from scipy.interpolate import griddata
from scipy.integrate import solve_ivp
from numpy.linalg import det
import pickle
from joblib import Parallel, delayed
import multiprocessing
import plotly as py
import pandas as pd
import numpy as np
import warnings
import sys
if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML

    # This is required for correct mathjax (latex) and documentation rendering
    HTML(
        """
        <script>
            var waitForPlotly = setInterval( function() {
                if( typeof(window.Plotly) !== "undefined" ){
                    MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
                    MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
                    clearInterval(waitForPlotly);
                }}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)
from colorlover import interp as color_interpolate
from plotly import tools
import plotly.graph_objs as go
import colorlover as cl
import math
import plotly.io as pio
pio.templates.default = "none"
import plotly.colors
from plotly.subplots import make_subplots
import pickle

colors = {'blues':['rgb(164,199,228)','rgb(119,171,214)','rgb(73,143,201)','rgb(49,113,165)','rgb(36,82,120)','rgb(23,52,76)'],
          'greens':['rgb(172,220,173)','rgb(130,203,133)','rgb(89,185,92)','rgb(63,152,66)','rgb(46,110,48)','rgb(28,67,29)'],
         'reds':['rgb(230,174,176)','rgb(218,133,137)','rgb(205,92,98)','rgb(187,57,64)','rgb(146,45,50)','rgb(104,32,35)'],
          #['rgb'+str(color) for color in plotly.colors.n_colors((254,224,210), (165,15,21), 6)],
         'pinks':['rgb(215,181,210)','rgb(195,144,188)','rgb(175,107,165)','rgb(147,79,137)','rgb(110,59,102)','rgb(74,40,69)'],
         'browns':['rgb(216,197,184)','rgb(196,168,148)','rgb(176,139,112)','rgb(150,111,83)','rgb(115,85,63)','rgb(80, 59, 44)'],
         'oranges': ['rgb(255,206,146)','rgb(255,189,109)','rgb(255,173,73)','rgb(255,157,36)','rgb(255, 140, 0)','rgb(219,120,0)'],
          'cyans': ['rgb(177,222,219)','rgb(139,205,201)','rgb(100,189,183)','rgb(70,163,157)','rgb(53,124,120)','rgb(37,87,84)'],
          'yellows': ['rgb(244,239,186)','rgb(239,231,151)','rgb(233,224,116)','rgb(228,216,82)','rgb(223,208,47)','rgb(200,185,31)',],
         'blacks':['rgb(182,182,182)','rgb(146,146,146)','rgb(109,109,109)','rgb(73,73,73)','rgb(36,36,36)','rgb(0,0,0)']}

colors_bg =  {'blues':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blues']],
              'greens':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['greens']],
              'reds':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['reds']],
              'pinks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['pinks']],
              'browns':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['browns']],
              'oranges':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['oranges']],
              'cyans': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['cyans']],
              'yellows': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['yellows']],
              'blacks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blacks']]}


def get_line_color(shade=2):
    return [color[shade] if shade<6 else color[5] for color in colors.values()]

def get_fill_color(shade=2):
    fill_colors = [color[shade] if shade<6 else color[5] for color in colors_bg.values()]


import numpy as np
import pandas as pd
import plotly.io as pio
from scipy.optimize import curve_fit


In [3]:
def get_result_from_list(results_list, kdeg_1=0.001, kdeg_2=0.001, param_space = np.logspace(-3,3,30), mode='closest'):
    param_space = np.array(list(set([result['params']['kdeg_1'] for result in results_list])))
    if mode=='closest':
        kdeg_1 = param_space[np.argmin(abs(param_space-kdeg_1))]
        kdeg_2 = param_space[np.argmin(abs(param_space-kdeg_2))]
    elif mode=='less':
        try:
            kdeg_1 = param_space[param_space-kdeg_1<0][-1]
        except IndexError:
            kdeg_1 = min(param_space)
        try:
            kdeg_2 = param_space[param_space-kdeg_2<0][-1]
        except IndexError:
            kdeg_2 = min(param_space)

    elif mode=='greater':
        try:
            kdeg_1 = param_space[param_space-kdeg_1>0][0]
        except IndexError:
            kdeg_1 = max(param_space)
        try:
            kdeg_2 = param_space[param_space-kdeg_2>0][0]
        except IndexError:
            kdeg_2 = max(param_space)
    temp = [result for result in results_list if result['params']['kdeg_1']==kdeg_1 and result['params']['kdeg_2']==kdeg_2]
    if len(temp)>1:
        raise('Duplicate entries found. Please check.')
    elif len(temp)==0:
        raise('No matching record found.')
    else:
        return dict(temp[0])

def plot_robustness(results):
    trace_list = []
    for i, result in enumerate(results):
        trace_list.append(go.Scatter(x=result['lower_x'], y=result['lower_y'], line=dict(color="rgba(12, 143, 190,1)"), 
                                     name='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)), 
                                     legendgroup='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)), 
                                     showlegend=False))
        trace_list.append(go.Scatter(x=result['upper_x'], y=result['upper_y'], fill='tonexty', line=dict(color="rgba(12, 143, 190,1)"),
                                     name='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)),
                                     legendgroup='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)),
                                     showlegend=False))


    layout = go.Layout(height=375, width=355, legend_orientation='v', legend_x = 1.05, legend_y = .95, legend_font = dict(family='Myriad Pro', size=14, color='black'),
                       xaxis=dict(title='kp_1(\u03bcM/h)',titlefont=dict(family='Myriad Pro', size=18, color='black'), title_standoff=0,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=4, tickangle=0, nticks=6, dtick=250,
                                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=True,zeroline=False,range=(0,1000)),
                       yaxis=dict(title='kp_2(\u03bcM/h)',
                                titlefont=dict(family='Myriad Pro', size=18, color='black'),title_standoff=0,
                                  anchor='x', side='left', showgrid=True, zeroline=False, nticks=6, dtick=250,
                                  tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                                  range=(0, 1000),showline=True, linewidth=1.25, linecolor='black', mirror=True, scaleanchor='x', scaleratio=1,
                                  ticks='outside', ticklen=4, tickangle=0))

    fig = go.Figure(data=trace_list, layout=layout)
    return fig

def plot_robustness_landscape(results, ticksize=18, height=460, width=510):
    color_scale=[[0, 'rgb(247, 253, 255)'],
           [0.5, "rgba(168, 228, 249,1)"],
           [0.9, "rgba(58, 194, 243,1)"],
           [1, "rgba(12, 143, 190,1)"]]

    z = [result['robustness']/1e6 if result['robustness']>0 else np.nan for result in results]
    param_space = sorted(list(set([result['params']['kdeg_1'] for result in results]).union(set([result['params']['kdeg_2'] for result in results]))))
    trace_list = []
    trace_list.append(go.Contour(x=param_space, y=param_space,
                                 z=np.array(z).reshape(30,30).transpose(), 
                                 colorscale=color_scale, zmin=1e-9, zmax=1, ncontours=10,
                                 contours=dict(showlabels=True, coloring='heatmap', labelfont=dict(family='Myriad Pro', size=13, color='black')),
                                 colorbar=dict(title='Robustness Metric<br>(Fraction of Parametric Space Bistable)', titleside='right', titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),
                                               thickness=16, len=0.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               tickvals = [1e-3,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55)))
    
#     y_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[i,:]))]
#     if y_lims:
#         max_y = max(param_space[:np.argmin(param_space<min(y_lims))])
#     else:
#         max_y = max(param_space)

#     x_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[:,i]))]
#     if x_lims:
#         max_x = max(param_space[:np.argmin(param_space<min(x_lims))])
#     else:
#         max_x = max(param_space)

    ax_range = (np.log10(min(param_space)), np.log10(max(param_space)))
    ax_type = 'log'

#     trace_list.append(go.Scatter(x=np.geomspace(min(param_space), max_x, 5), y=[max_y]*5, name='limits', showlegend=False, mode='lines',
#                                  line=dict(color='black', dash='dash', width=0.5)))
#     trace_list.append(go.Scatter(y=np.geomspace(min(param_space), max_y, 5), x=[max_x]*5, name='limits', showlegend=False, mode='lines',
#                                      line=dict(color='black', dash='dash', width=0.5)))

    layout = go.Layout(height=height, width=width,plot_bgcolor='rgba(207,28,43,0.15)',
                       xaxis=dict(title='k<sub>deg1</sub> (1/h)', 
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=4, tickangle=-90, nticks=12, type=ax_type, dtick='D2',
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=False,zeroline=False,range=ax_range),
                      yaxis=dict(title='k<sub>deg2</sub> (1/h)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,

                                  anchor='x', side='left', showgrid=False, zeroline=False, type=ax_type, dtick='D2',
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                scaleanchor='x', scaleratio=1, 
                                  range=ax_range,showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=4, tickangle=0))

    fig = go.Figure(data=trace_list, layout=layout)
    return fig

    
def plot_speed_landscape(results, ticksize=18, height=460, width=510):
    colors = [[0, "rgba(248, 253, 246, 1)"],
              [0.33, "rgba(214, 249, 199, 1)"],
              [0.66, "rgba(141, 240, 100, 1)"],
              [1.0, "rgba(59, 161, 16, 1)"]]
    points = ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']
    x = [result['params']['kdeg_1'] for result in results]
    y = [result['params']['kdeg_2'] for result in results]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        avg_fast_times_1 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        std_fast_times_1 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        avg_fast_times_2 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        std_fast_times_2 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        avg_fast_times = np.nanmean([avg_fast_times_1, avg_fast_times_2],axis=0)
        std_fast_times = [np.sqrt(np.nanmean([std_1**2, std_2**2], axis=0)*sum(~np.isnan([std_1**2, std_2**2]))) for std_1, std_2 in zip(std_fast_times_1, std_fast_times_2)]
    z = np.log10(1/avg_fast_times).reshape(30,30).transpose()
    param_space = sorted(list(set([result['params']['kdeg_1'] for result in results]).union(set([result['params']['kdeg_2'] for result in results]))))
    
    trace_list = []
    trace_list.append(go.Contour(x=param_space, y=param_space,
                                 z=z, ncontours=20, zmin=-4, zmax=2,
                                 colorscale=colors, 
                                 contours=dict(showlabels=True, coloring='heatmap', labelfont=dict(family='Myriad Pro', size=13, color='black')),#, start=-1, end=4),
                                 colorbar=dict(title='Speed Metric<br>(1/Time to Switch (h))', titleside='right', titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), #nticks=30,
                                               tickvals = np.linspace(-4,2,7),
                                               tickprefix='10<sup>', ticksuffix='</sup>',
                                               thickness=16, len=0.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55)))
    ax_range = (np.log10(min(param_space)), np.log10(max(param_space)))
    ax_type = 'log'

#     y_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[i,:]))]
#     if y_lims:
#         max_y = max(param_space[:np.argmin(param_space<min(y_lims))])
#     else:
#         max_y = max(param_space)

#     x_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[:,i]))]
#     if x_lims:
#         max_x = max(param_space[:np.argmin(param_space<min(x_lims))])
#     else:
#         max_x = max(param_space)


    layout = go.Layout(height=height, width=width,plot_bgcolor='rgba(207,28,43,0.15)',
                       coloraxis=dict(colorbar_nticks=20, colorbar=dict(dtick='D2')),
                       xaxis=dict(title='k<sub>deg1</sub> (1/h)', dtick='D2',
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=4, tickangle=-90, nticks=12, type=ax_type,
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=False,zeroline=False,range=ax_range),
                      yaxis=dict(title='k<sub>deg2</sub> (1/h)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,
                                  anchor='x', side='left', showgrid=False, zeroline=False, type=ax_type,
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                scaleanchor='x', scaleratio=1, dtick='D2',
                                  range=ax_range,showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=4, tickangle=0))

    speed_landscape = go.Figure(data=trace_list, layout=layout)
    
    stdev_landscape = go.Figure(data=[go.Histogram(x=np.array(std_fast_times)*100/np.array(avg_fast_times), histnorm='probability', xbins=dict(size=1, start=0),
                                      marker=dict(color='rgba(207,28,43,0.55)',line_color='black', line_width=0))], 
                                  layout=go.Layout(height=400, width=375, bargap=0.2,

                                               xaxis=dict(title='% Standard Deviation/Mean',
                                                          titlefont=dict(family='Myriad Pro', size=16, color='black'), title_standoff=0.2,
                                                          showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                                          ticks='outside', ticklen=4, tickangle=0, nticks=13, type='linear',
                                                          tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                                                          showgrid=False,zeroline=False,range=(0,12)),

                                              yaxis=dict(title='Fraction of Points',
                                                        titlefont=dict(family='Myriad Pro', size=16, color='black'),title_standoff=0.2,
                                                          anchor='x', side='left', showgrid=True, zeroline=False, type='linear', nticks=12,
                                                        tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                                                          range=(0,1),showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                                          ticks='outside', ticklen=4, tickangle=0)))
    return speed_landscape, stdev_landscape


def plot_speed_vs_robustness(rob_results, speed_results, start=3, offset=2, label_locs='auto',ticksize=18, height=460, width=510):
    color_scale = py.colors.diverging.RdBu[0:4]+ py.colors.diverging.RdBu[-4:]
    rob_results = rob_results
    speed_results = speed_results
    points = ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        avg_fast_times_1 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        std_fast_times_1 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        avg_fast_times_2 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        std_fast_times_2 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        avg_fast_times = np.nanmean([avg_fast_times_1, avg_fast_times_2],axis=0)
        std_fast_times = [np.sqrt(np.nanmean([std_1**2, std_2**2], axis=0)*sum(~np.isnan([std_1**2, std_2**2]))) for std_1, std_2 in zip(std_fast_times_1, std_fast_times_2)]
    results_dict = dict(robustness=[result['robustness']/1e6 for result in rob_results],
                        speed=avg_fast_times, speed_std = std_fast_times,
                        kdeg_1 = [result['params']['kdeg_1'] for result in rob_results],
                        kdeg_2 = [result['params']['kdeg_2'] for result in rob_results],
                        ratio = [result['params']['kdeg_1']/result['params']['kdeg_2'] for result in speed_results])
    df = pd.DataFrame(data=results_dict)
    df = df.dropna()
    df['rounded_ratio']=df['ratio'].apply(lambda x: np.round(x, decimals=-int(np.floor(np.log10(x)))+2))
    df = df[df['robustness']>=0.001]
    opt_ratio = df[df['speed']>np.round(max(df['speed']),decimals=-int(np.floor(np.log10(max(df['speed']))))+1)].sort_values(by=['robustness'],ascending=False).iloc[0]['rounded_ratio']
    invisible_ratios = [ratio for ratio in sorted(list(set(df['rounded_ratio']))) if sum(1/df[df['rounded_ratio']==ratio]['speed'] > 0.001) < 3]
    temp_df = df[~df['rounded_ratio'].isin(invisible_ratios)]
    ratios = sorted(list(set(temp_df['rounded_ratio'])))
    opt_ratio_location = np.where(ratios==opt_ratio)[0][0]
    ratios = ratios[opt_ratio_location::-2][::-1] + ratios[opt_ratio_location+start::offset][:]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=temp_df['robustness'],
                                 x=1/temp_df['speed'],
                                 mode='markers', showlegend=False,
                                 marker=dict(color=np.log10(temp_df['rounded_ratio']), size=0.0001,
                                             colorscale=color_scale, line=dict(color='black', width=0.15),
                                             colorbar=dict(title='r = k<sub>deg1</sub>/k<sub>deg2</sub><br>(Ratio of Degradation Rates)', titleside='right', nticks=12,
                                                           titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),
                                               tickprefix='10<sup>', ticksuffix='</sup>',
                                               thickness=16, len=.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55))))

    for i, ratio in enumerate(ratios):
        if np.log10(ratio)>=-3:
            if np.log10(ratio)>=0:
                text = str(int(np.round(ratio, decimals=-int(np.floor(np.log10(ratio))))))
            else:
                text = str((np.round(ratio, decimals=-int(np.floor(np.log10(ratio))))))
        else:
            text='{:.1e}'.format(ratio).replace("e-0", "e-")
        temp = temp_df[temp_df['rounded_ratio']==ratio].sort_values(by=['speed'], ascending=False)
        color_index = (np.log10(ratio) - np.log10(min(temp_df['rounded_ratio'])))/ (np.log10(max(temp_df['rounded_ratio']))- np.log10(min(temp_df['rounded_ratio']))) * (len(color_scale)-1)
        if np.isnan(color_index):
            color_index = 0
        lower_index = int(color_index)
        upper_index = lower_index + 1
        if lower_index >= (len(color_scale) - 1):
            color = color_scale[-1]
        elif upper_index < 1:
            color = color_scale[0]
        else:
            color = py.colors.find_intermediate_color(color_scale[lower_index], color_scale[upper_index], color_index - int(color_index), colortype='rgb')
        if ratio==opt_ratio:
            dash='dot'
            showlegend=True
        else:
            dash=None
            showlegend=False

        if min(temp['robustness']>=0.02):
            fig.add_trace(go.Scatter(y=[temp['robustness'].iloc[-1]], x=[1/temp['speed'].iloc[-1]], mode='markers', marker=dict(symbol='x',color=color, size=7, line=dict(color='black', width=1)),showlegend=False))

        fig.add_trace(go.Scatter(x=1/temp['speed'], y=temp['robustness'], name='r='+text, 
                                     mode='lines',  line=dict(color=color, shape='spline', smoothing=1.3, dash=dash), 
                                     showlegend=showlegend))
        
        x = np.log10((1/temp[1/temp['speed']>0.001]['speed'])).values
        y = temp[1/temp['speed']>0.001]['robustness'].values
        mid_y = (max(y) + min(y))/2
        mid_y_index = np.argmin(abs(y-mid_y))

        if min(temp['robustness'])>=.2 and ratio!=opt_ratio:
            if label_locs == 'auto':
                yshift=-9
                xshift=-9
            else:
                xshift=4
                yshift=4
            fig.add_annotation(x=np.log10((1/temp[1/temp['speed']>0.001]['speed']).iloc[-1]),
                               y=(temp[1/temp['speed']>0.001]['robustness']).iloc[-1],
                               text='r='+text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[-1]),
                               font=dict(family='Myriad Pro', size=13, color=color), yshift=yshift,xshift=xshift)
        elif (max(y)-min(y)) > 0.1 and ratio<opt_ratio:
            if label_locs=='auto':
                yshift=-6
                xshift=-6
            else:
                xshift=4
                yshift=4
            fig.add_annotation(x=x[mid_y_index],
                   y=y[mid_y_index],
                   text='r='+text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[mid_y_index]),
                   font=dict(family='Myriad Pro', size=13, color=color), yshift=yshift,xshift=xshift)
        elif (max(y)-min(y)) > 0.1 and ratio>=opt_ratio:
            yshift=4
            xshift=4
            fig.add_annotation(x=x[mid_y_index],
                   y=y[mid_y_index],
                   text='r='+text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[mid_y_index]),
                   font=dict(family='Myriad Pro', size=13, color=color), yshift=yshift,xshift=xshift)


    fig.update_layout(height=500, width=510,
                     legend_font=dict(family='Myriad Pro', size=ticksize, color='black'), legend_y=0.01,
                       xaxis=dict(title='Speed Metric<br>(1/Time to Switch (h))',
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=4, tickangle=-90, nticks=12, type='log', dtick='D2',
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=True,zeroline=False,range=(-3,2)),

                      yaxis=dict(title='Robustness Metric<br>(Fraction of Parametric Space Bistable)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,
                                nticks=12,
                                  anchor='x', side='left', showgrid=True, zeroline=False, type='linear',
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                 scaleanchor='x', scaleratio=1,
                                  range=(0,1),showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=4, tickangle=0))

    return fig

In [4]:
def simple_toggle_ode(concs, params):
    lac_i, tet_r = concs
    
    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']
    
    dlaci_dt = kp_1/(1+(tet_r/kdiss_1)**n_1) - kdeg_1*lac_i
    dtetr_dt = kp_2/(1+(lac_i/kdiss_2)**n_2) - kdeg_2*tet_r

    return [dlaci_dt, dtetr_dt]    



def simple_toggle_jacobian(concs, params):
    lac_i, tet_r = concs

    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']

    return [[-kdeg_1, -(kp_1*n_1/kdiss_1**n_1)*(tet_r**(n_1-1)/(1+(tet_r/kdiss_1)**n_1)**2)],
            [-(kp_2*n_2/kdiss_2**n_2)*(lac_i**(n_2-1)/(1+(lac_i/kdiss_2)**n_2)**2),  -kdeg_2 ]]
        
    
    
def simple_toggle_nullclines(conc_ranges, params):
    lac_range, tet_range = conc_ranges
    
    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']
    lac_null = kp_1/(kdeg_1*(1+(tet_range/kdiss_1)**n_1))
    tet_null = kp_2/(kdeg_2*(1+(lac_range/kdiss_2)**n_2))
    
    return lac_null, tet_null

In [5]:
def invert_robustness_result(result):
    reverse = {'2':'1', '1':'2', 'x':'y', 'y':'x'}
    inverse = {key.replace(key.split('_')[-1], reverse[key.split('_')[-1]]):result[key] for key in ['lower_x', 'lower_y', 'upper_x', 'upper_y', 'raw_lower_x', 'raw_upper_x', 'raw_lower_y', 'raw_upper_y']}
    inverse['params']={key.replace(key.split('_')[-1], reverse[key.split('_')[-1]]):result['params'][key] for key in result['params']}
    inverse['robustness'] = result['robustness']
    inverse['status'] = result['status']
    return inverse

def invert_robustness_results_list(results_list):
    temp_list = [invert_robustness_result(result) for result in results_list]
    temp_list = sorted(temp_list, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))
    return temp_list

def invert_speed_result(result):
    reverse = {'2':'1', '1':'2', 'x':'y', 'y':'x', 'lac':'tet', 'tet':'lac', 'top_left_corner':'top_right_corner', 'top_right_corner':'top_left_corner', 'top_corner':'top_corner', 'bottom_corner':'bottom_corner'}
    points = ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']
    switch_types = ['fast', 'slow', 'deg_mediated']
    inverse = {key:None for key in result}
    inverse['params']={key.replace(key.split('_')[-1], reverse[key.split('_')[-1]]):result['params'][key] for key in result['params']}
    for point in points:
        inverse[reverse[point]] = {key.replace(key.split('_')[-1], reverse[key.split('_')[-1]]):result[point][key] for key in ['kp_1', 'kp_2']}
        if result[point]['switch_times']:
            inverse[reverse[point]]['switch_times'] = {switch: {reverse[key]:result[point]['switch_times'][switch][key] for key in ['lac', 'tet']} for switch in switch_types}
        else:
            inverse[reverse[point]]['switch_times'] = None
    return inverse

def invert_speed_results_list(results_list):
    temp_list = [invert_speed_result(result) for result in results_list]
    temp_list = sorted(temp_list, key=lambda x: (x['params']['kdeg_1'], x['params']['kdeg_2']))
    return temp_list


The below block of code plots all figures for one parameter set

In [6]:
file_name = "C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/robustness/1/Kd_1_1__n_2_2.pickle" 
with open(file_name, 'rb') as handle:
    rob_results_list = pickle.load(handle)
    
file_name = "C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/speed/1/Kd_1_1__n_2_2_speed.pickle" 
with open(file_name, 'rb') as handle:
    speed_results_list = pickle.load(handle)
    


In [7]:
rob_landscape_fig = plot_robustness_landscape(rob_results_list, height=475, width=522.5)
plot(rob_landscape_fig)
rob_landscape_fig.write_image('robustness_landscape.svg', format='svg')
speed_landscape_fig, speed_stdev_fig = plot_speed_landscape(speed_results_list, height=475, width=514)
plot(speed_landscape_fig)
plot(speed_stdev_fig)
#speed_landscape_fig.write_image('speed_landscape.svg', format='svg')
#speed_stdev_fig.write_image('speed_stdev.svg', format='svg')
speed_vs_robustness_fig = plot_speed_vs_robustness(rob_results_list, speed_results_list, label_locs='no', start=2)
plot(speed_vs_robustness_fig)
#speed_vs_robustness_fig.write_image('speed_vs_robustness.svg', format='svg')



In [769]:
def miniplot_robustness_landscape(results, ticksize=26, height=350, width=330):
    color_scale=[[0, 'rgb(247, 253, 255)'],
           [0.5, "rgba(168, 228, 249,1)"],
           [0.9, "rgba(58, 194, 243,1)"],
           [1, "rgba(12, 143, 190,1)"]]

    z = [result['robustness']/1e6 if result['robustness']>0 else np.nan for result in results]
    param_space = sorted(list(set([result['params']['kdeg_1'] for result in results]).union(set([result['params']['kdeg_2'] for result in results]))))
    trace_list = []
    trace_list.append(go.Contour(x=param_space, y=param_space,
                                 z=np.array(z).reshape(30,30).transpose(), 
                                 colorscale=color_scale, zmin=1e-9, zmax=1, showscale=False, autocontour=False,
                                 contours=dict(showlabels=True, coloring='heatmap', labelfont=dict(family='Myriad Pro', size=18, color='black'), start=0.1, size=0.2, end=0.9),
                                 colorbar=dict(title='Robustness Metric<br>(Fraction of Parametric Space Bistable)', titleside='right', titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),
                                               thickness=16, len=0.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               tickvals = [0.1, 0.2, 0.4, 0.6,0.8, 0.9], 
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55)))

    ax_range = (np.log10(min(param_space)), np.log10(max(param_space)))
    ax_type = 'log'


    layout = go.Layout(height=height, width=width,plot_bgcolor='rgba(207,28,43,0.15)',
                       xaxis=dict(#title='k<sub>deg1</sub> (1/h)', 
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=8, tickangle=0, nticks=12, type=ax_type, dtick=2,tick0=-3,
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=False,zeroline=False,range=ax_range),
                      yaxis=dict(#title='k<sub>deg2</sub> (1/h)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,

                                  anchor='x', side='left', showgrid=False, zeroline=False, type=ax_type,  dtick=2,tick0=-3,
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                scaleanchor='x', scaleratio=1, 
                                  range=ax_range,showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=8, tickangle=-90))

    fig = go.Figure(data=trace_list, layout=layout)
    return fig


def miniplot_speed_landscape(results, ticksize=26, height=350, width=412):
    colors = [[0, "rgba(248, 253, 246, 1)"],
              [0.33, "rgba(214, 249, 199, 1)"],
              [0.66, "rgba(141, 240, 100, 1)"],
              [1.0, "rgba(59, 161, 16, 1)"]]
    points = ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']
    x = [result['params']['kdeg_1'] for result in results]
    y = [result['params']['kdeg_2'] for result in results]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        avg_fast_times_1 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        std_fast_times_1 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        avg_fast_times_2 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        std_fast_times_2 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in results]
        avg_fast_times = np.nanmean([avg_fast_times_1, avg_fast_times_2],axis=0)
        std_fast_times = [np.sqrt(np.nanmean([std_1**2, std_2**2], axis=0)*sum(~np.isnan([std_1**2, std_2**2]))) for std_1, std_2 in zip(std_fast_times_1, std_fast_times_2)]
    z = np.log10(1/avg_fast_times).reshape(30,30).transpose()
    param_space = sorted(list(set([result['params']['kdeg_1'] for result in results]).union(set([result['params']['kdeg_2'] for result in results]))))
    
    trace_list = []
    trace_list.append(go.Contour(x=param_space, y=param_space,
                                 z=z, zmin=-4, zmax=2,
                                 colorscale=colors, showscale=True, autocontour=False, line=dict(width=0.75),
                                 contours=dict(showlabels=True, coloring='heatmap', labelfont=dict(family='Myriad Pro', size=18, color='black'), start=-3, end=1, size=1),
                                 colorbar=dict(title='Speed Metric<br>(1/Time to Switch (h))', titleside='right', titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), #nticks=30,
                                               tickvals = np.linspace(-4,2,7),
                                               tickprefix='10<sup>', #ticksuffix='</sup>',
                                               thickness=16, len=0.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55)))
    ax_range = (np.log10(min(param_space)), np.log10(max(param_space)))
    ax_type = 'log'

#     y_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[i,:]))]
#     if y_lims:
#         max_y = max(param_space[:np.argmin(param_space<min(y_lims))])
#     else:
#         max_y = max(param_space)

#     x_lims = [param_space[i] for i, vals in enumerate(robs) if all(np.isnan(robs[:,i]))]
#     if x_lims:
#         max_x = max(param_space[:np.argmin(param_space<min(x_lims))])
#     else:
#         max_x = max(param_space)


    layout = go.Layout(height=height, width=width,plot_bgcolor='rgba(207,28,43,0.15)',
                       xaxis=dict(#title='k<sub>deg1</sub> (1/h)', 
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=8, tickangle=0, nticks=12, type=ax_type, dtick=2,tick0=-3,
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=False,zeroline=False,range=ax_range),
                      yaxis=dict(#title='k<sub>deg2</sub> (1/h)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,

                                  anchor='x', side='left', showgrid=False, zeroline=False, type=ax_type,  dtick=2,tick0=-3,
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                scaleanchor='x', scaleratio=1, 
                                  range=ax_range,showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=8, tickangle=-90))


    speed_landscape = go.Figure(data=trace_list, layout=layout)
    stdevs = np.array(std_fast_times)*100/np.array(avg_fast_times)
    max_stdev = (1-(sum(stdevs > 5)/len(stdevs)))*100
    stdev_landscape = go.Figure(data=[go.Histogram(x=np.array(std_fast_times)*100/np.array(avg_fast_times), histnorm='probability', xbins=dict(size=1, start=0),
                                      marker=dict(color='rgba(207,28,43,0.55)',line_color='black', line_width=0))], 
                                  layout=go.Layout(height=350, width=330, bargap=0.2,

                                               xaxis=dict(#title='% Standard Deviation/Mean',
                                                          titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                                          showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                                          ticks='outside', ticklen=8, tickangle=0, nticks=10, type='linear',
                                                          tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                                          showgrid=False,zeroline=False,range=(0,12)),

                                              yaxis=dict(#title='Fraction of Points',
                                                        titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,
                                                          anchor='x', side='left', showgrid=True, zeroline=False, type='linear', nticks=6,
                                                        tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 

                                                          range=(0,1),showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                                          ticks='outside', ticklen=8, tickangle=0)))
    return speed_landscape, stdev_landscape, max_stdev




def miniplot_speed_vs_robustness(rob_results, speed_results, start=3, offset=3, label_locs='auto',ticksize=26, height=310, width=412):
    color_scale = py.colors.diverging.RdBu[0:4]+ py.colors.diverging.RdBu[-4:]
    rob_results = rob_results
    speed_results = speed_results
    points = ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        avg_fast_times_1 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        std_fast_times_1 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['lac'])))*result[point]['switch_times']['fast']['lac'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        avg_fast_times_2 = [np.nanmean([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        std_fast_times_2 = [np.nanstd([int(not(np.isinf(result[point]['switch_times']['fast']['tet'])))*result[point]['switch_times']['fast']['tet'] if result[point]['switch_times'] else np.nan for point in points]) for result in speed_results]
        avg_fast_times = np.nanmean([avg_fast_times_1, avg_fast_times_2],axis=0)
        std_fast_times = [np.sqrt(np.nanmean([std_1**2, std_2**2], axis=0)*sum(~np.isnan([std_1**2, std_2**2]))) for std_1, std_2 in zip(std_fast_times_1, std_fast_times_2)]
    results_dict = dict(robustness=[result['robustness']/1e6 for result in rob_results],
                        speed=avg_fast_times, speed_std = std_fast_times,
                        kdeg_1 = [result['params']['kdeg_1'] for result in rob_results],
                        kdeg_2 = [result['params']['kdeg_2'] for result in rob_results],
                        ratio = [result['params']['kdeg_1']/result['params']['kdeg_2'] for result in speed_results])
    df = pd.DataFrame(data=results_dict)
    df = df.dropna()
    df['rounded_ratio']=df['ratio'].apply(lambda x: np.round(x, decimals=-int(np.floor(np.log10(x)))+2))
    df = df[df['robustness']>=0.001]
    #opt_ratio = df[df['speed']>np.round(max(df['speed']),decimals=-int(np.floor(np.log10(max(df['speed']))))+1)].sort_values(by=['robustness'],ascending=False).iloc[0]['rounded_ratio']
    opt_ratio_est = rob_results[0]['params']['kdiss_1']/rob_results[0]['params']['kdiss_2']
    opt_ratio = sorted(list(set(df['rounded_ratio'])))[np.argmin(abs(np.array(sorted(list(set(df['rounded_ratio']))))-opt_ratio_est))]
    invisible_ratios = [ratio for ratio in sorted(list(set(df['rounded_ratio']))) if sum(1/df[df['rounded_ratio']==ratio]['speed'] > 0.001) < 2]
    temp_df = df[~df['rounded_ratio'].isin(invisible_ratios)]
    temp_df=df
    ratios = np.array(sorted(list(set(temp_df['rounded_ratio']))))
    opt_ratio_location = np.where(ratios==opt_ratio)[0][0]
    ratios = list(ratios[opt_ratio_location::-3][::-1]) + list(ratios[opt_ratio_location+2::3][:])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=temp_df['robustness'],
                                 x=1/temp_df['speed'],
                                 mode='markers', showlegend=False,
                                 marker=dict(color=np.log10(temp_df['rounded_ratio']), size=0.0001, cmin=-3, cmax=3,
                                             colorscale=color_scale, line=dict(color='black', width=0.15),
                                             colorbar=dict(title='r = k<sub>deg1</sub>/k<sub>deg2</sub><br>(Ratio of Degradation Rates)', titleside='right', nticks=12,
                                                           titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),
                                               tickprefix='10<sup>', ticksuffix='</sup>',
                                               thickness=16, len=.9, ticks='outside', tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                               outlinecolor='black', outlinewidth=1.25, exponentformat='power', y=0.55))))

    for i, ratio in enumerate(ratios):
        if np.log10(ratio)>=-3:
            if np.log10(ratio)>=0:
                text = str(int(np.round(ratio, decimals=-int(np.floor(np.log10(ratio))))))
            else:
                text = str((np.round(ratio, decimals=-int(np.floor(np.log10(ratio))))))
        else:
            text='{:.1e}'.format(ratio).replace("e-0", "e-")
        temp = temp_df[temp_df['rounded_ratio']==ratio].sort_values(by=['speed'], ascending=False)
        color_index = (np.log10(ratio) - (-3))/ (6) * (len(color_scale)-1)
        if np.isnan(color_index):
            color_index = 0
        if color_index < 0:
            color = color_scale[0]
        else:
            lower_index = int(color_index)
            upper_index = lower_index + 1
            if lower_index >= (len(color_scale) - 1):
                color = color_scale[-1]
            elif upper_index < 1:
                color = color_scale[0]
            else:
                color = py.colors.find_intermediate_color(color_scale[lower_index], color_scale[upper_index], color_index - int(color_index), colortype='rgb')
        if ratio==opt_ratio:
            dash='dot'
            showlegend=False
            color='black'
        else:
            dash=None
            showlegend=False

        if min(temp['robustness']>=0.02):
            fig.add_trace(go.Scatter(y=[temp['robustness'].iloc[-1]], x=[1/temp['speed'].iloc[-1]], mode='markers', marker=dict(symbol='x',color=color, size=7, line=dict(color='black', width=1)),showlegend=False))

        fig.add_trace(go.Scatter(x=1/temp['speed'], y=temp['robustness'], name='r='+text, 
                                     mode='lines',  line=dict(color=color, shape='spline', smoothing=1.3, dash=dash), 
                                     showlegend=showlegend))
        
        x = np.log10((1/temp[1/temp['speed']>0.001]['speed'])).values
        y = temp[1/temp['speed']>0.001]['robustness'].values
        if len(y)>2:
            mid_y = (max(y) + min(y))/2
            mid_y_index = np.argmin(abs(y-mid_y))

            if min(temp['robustness'])>=.2 and ratio!=opt_ratio:
                if label_locs == 'auto':
                    yshift=-10
                    xshift=-10
                else:
                    xshift=5
                    yshift=5
                fig.add_annotation(x=np.log10((1/temp[1/temp['speed']>0.001]['speed']).iloc[-1]),
                                   y=(temp[1/temp['speed']>0.001]['robustness']).iloc[-1],
                                   text=text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[-1]),
                                   font=dict(family='Myriad Pro', size=18, color=color), yshift=yshift,xshift=xshift)
            elif (max(y)-min(y)) > 0.1 and ratio<opt_ratio:
                if label_locs=='auto':
                    yshift=-9
                    xshift=-9
                else:
                    xshift=5
                    yshift=5
                fig.add_annotation(x=x[mid_y_index],
                       y=y[mid_y_index],
                       text=text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[mid_y_index]),
                       font=dict(family='Myriad Pro', size=18, color=color), yshift=yshift,xshift=xshift)
            elif (max(y)-min(y)) > 0.1 and ratio>=opt_ratio:
                yshift=5
                xshift=5
                fig.add_annotation(x=x[mid_y_index],
                       y=y[mid_y_index],
                       text=text, showarrow=False, textangle=((-180*np.arctan(5/1.05*np.gradient(temp[1/temp['speed']>0.001]['robustness'])/np.gradient(np.log10(1/(temp[1/temp['speed']>0.001]['speed']))))/np.pi)[mid_y_index]),
                       font=dict(family='Myriad Pro', size=18, color=color), yshift=yshift,xshift=xshift)


    fig.update_layout(height=height, width=width,
                     legend_font=dict(family='Myriad Pro', size=ticksize, color='black'), legend_y=0.01,
                       xaxis=dict(#title='Speed Metric<br>(1/Time to Switch (h))',
                                  titlefont=dict(family='Myriad Pro', size=ticksize, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=8, tickangle=0, nticks=12, type='log', dtick=1,
                                  tickfont=dict(size=ticksize, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=True,zeroline=False,range=(-3,2)),

                      yaxis=dict(#title='Robustness Metric<br>(Fraction of Parametric Space Bistable)',
                                titlefont=dict(family='Myriad Pro', size=ticksize, color='black'),title_standoff=0.2,
                                nticks=7,
                                  anchor='x', side='left', showgrid=True, zeroline=False, type='linear',
                                tickfont=dict(family='Myriad Pro',size=ticksize, color='black'), tickcolor='black', 
                                 scaleanchor='x', scaleratio=1,
                                  range=(0,1),showline=True, linewidth=1.25, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=8, tickangle=0))

    return fig,opt_ratio


In [770]:
file_name = "C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/robustness/0005/Kd_500_0005__n_2_2.pickle" 
with open(file_name, 'rb') as handle:
    rob_results_list = pickle.load(handle)
    #rob_results_list = invert_robustness_results_list(rob_results_list)
file_name = "C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/speed/0005/Kd_500_0005__n_2_2_speed.pickle" 
with open(file_name, 'rb') as handle:
    speed_results_list = pickle.load(handle)
    #speed_results_list = invert_speed_results_list(speed_results_list)
    fig,df = miniplot_speed_vs_robustness(rob_results_list, speed_results_list, width=410, height=350)
plot(fig)
#fig1.write_image('speed_vs_robustness_small.svg', format='svg')


In [748]:
df

92400.0

In [715]:
plot_robustness_landscape(rob_results_list)

In [771]:
KD_list = ['0.005', '0.01', '0.1', '1', '10', '100', '500']
KD_values = [0.005, 0.01, 0.1, 1, 10, 100, 500]

results_dict = {str(i):[] for i in KD_list}
speed_prefix = 'C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/speed/'
robustness_prefix = 'C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Models/robustness/'
file_prefix = 'Kd_'
speed_suffix = '__n_2_2_speed.pickle'
robustness_suffix = '__n_2_2.pickle'
KD_2_list = [[KD]*(7-i) for i,KD in enumerate(KD_list)]
KD_2_list = [KD for KDlist in KD_2_list for KD in KDlist]
KD_1_list = [KD_list[i:] for i,KD in enumerate(KD_list)]
KD_1_list = [KD for KDlist in KD_1_list for KD in KDlist]

In [284]:
KD_1_list = [[KD]*(7-i) for i,KD in enumerate(KD_list)]
KD_1_list = [KD for KDlist in KD_1_list for KD in KDlist]

In [773]:

for KD1, KD2 in zip(KD_1_list, KD_2_list):
    speed_directory = speed_prefix + KD1.replace('.', '')+'/'
    robustness_directory = robustness_prefix + KD1.replace('.', '')+'/'
    speed_file_name = file_prefix+KD1.replace('.','')+'_'+KD2.replace('.','')+speed_suffix
    robustness_file_name = file_prefix+KD1.replace('.','')+'_'+KD2.replace('.','')+robustness_suffix
    if speed_file_name in os.listdir(speed_directory):
        with open(speed_directory+speed_file_name, 'rb') as handle:
            speed_results_list = pickle.load(handle)
        with open(robustness_directory+robustness_file_name, 'rb') as handle:
            rob_results_list = pickle.load(handle)
        fig1,opt_ratio = miniplot_speed_vs_robustness(rob_results_list, speed_results_list, width=410, height=350)
        print(KD2, KD1)
        plot(fig1)
        fig1.write_image('speed_vs_robustness_KD1_'+KD2+'_KD2_'+KD1+'_ratio_'+str(opt_ratio)+'.svg', format='svg')
    else:
        speed_file_name = file_prefix+KD2.replace('.','')+'_'+KD1.replace('.','')+speed_suffix
        robustness_file_name = file_prefix+KD2.replace('.','')+'_'+KD1.replace('.','')+robustness_suffix
        with open(speed_directory+speed_file_name, 'rb') as handle:
            temp_results_list = pickle.load(handle)
            speed_results_list = invert_speed_results_list(temp_results_list)

        with open(robustness_directory+robustness_file_name, 'rb') as handle:
            temp_results_list = pickle.load(handle)
            rob_results_list = invert_robustness_results_list(temp_results_list)
        fig1, opt_ratio = miniplot_speed_vs_robustness(rob_results_list, speed_results_list, width=410, height=350)
        print(KD2, KD1)

        plot(fig1)
        fig1.write_image('speed_vs_robustness_KD1_'+KD2+'_KD2_'+KD1+'_ratio_'+str(opt_ratio)+'.svg', format='svg')


0.005 0.005


0.005 0.01


0.005 0.1


0.005 1


0.005 10


...

10 100


10 500


100 100


100 500


500 500


In [210]:
import colorlover as cl
from IPython.display import HTML
display(HTML(cl.to_html( py.colors.diverging.RdBu[1:5]+py.colors.diverging.RdBu[-4:])))